In [1]:
%pip install -q kagglehub
import kagglehub
import tensorflow as tf
import cv2
import numpy as np
import os

# Download latest version of the MoveNet Thunder model
model_dir = kagglehub.model_download("google/movenet/tensorFlow2/singlepose-thunder")
# model_dir will be something like:
#   "C:/Users/YourUserName/.cache/kagglehub/models/google/movenet/tensorFlow2/singlepose-thunder"

# Sometimes MoveNet model files are stored under a subdirectory named "4"
# Check if "4" exists in model_dir; if so, point model_path to that subdirectory:
model_path = os.path.join(model_dir, "4")
if not os.path.exists(model_path):
    # If there's no '4' subfolder, we can just use model_dir directly:
    model_path = model_dir

# Load the saved model
model = tf.saved_model.load(model_path)
movenet = model.signatures['serving_default']






[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip
c:\Users\assaf\Documents\code\neural\final\tf-env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import numpy as np
import cv2
import tensorflow as tf

def preprocess_frame(frame):
    img = tf.image.resize_with_pad(tf.expand_dims(frame, axis=0), 256, 256)
    img = tf.cast(img, dtype=tf.int32)
    return img

def extract_keypoints(frame):
    input_image = preprocess_frame(frame)
    outputs = movenet(input_image)
    keypoints = outputs['output_0'].numpy()
    flat_input = keypoints[0, 0, :, :2].flatten()  # (34,)
    return flat_input

# Set paths
video_root_dir = './videos'
output_root_dir = './position_files'
os.makedirs(output_root_dir, exist_ok=True)

# Walk through label subdirectories
for label in os.listdir(video_root_dir):
    label_video_dir = os.path.join(video_root_dir, label)
    if not os.path.isdir(label_video_dir):
        continue  # Skip files

    # Create matching output subdirectory
    label_output_dir = os.path.join(output_root_dir, label)
    os.makedirs(label_output_dir, exist_ok=True)

    # Process each video in that subdirectory
    for filename in os.listdir(label_video_dir):
        if filename.endswith('.mp4'):
            video_path = os.path.join(label_video_dir, filename)
            cap = cv2.VideoCapture(video_path)

            data = []

            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    break

                rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                flat_input = extract_keypoints(rgb)
                data.append(flat_input)

            cap.release()

            data = np.array(data)  # (num_frames, 34)
            output_path = os.path.join(label_output_dir, f"{os.path.splitext(filename)[0]}.npy")
            np.save(output_path, data)
            print(f"Saved: {output_path}")

cv2.destroyAllWindows()

Saved: ./position_files\good_right\63009_4.npy
Saved: ./position_files\good_right\63043_1.npy
Saved: ./position_files\good_right\63094_1.npy
Saved: ./position_files\good_right\63178_1.npy
Saved: ./position_files\good_right\63208_2.npy
Saved: ./position_files\good_right\63240_2.npy
Saved: ./position_files\good_right\63241_1.npy
Saved: ./position_files\good_right\63274_5.npy
Saved: ./position_files\good_right\63538_1.npy
Saved: ./position_files\good_right\63687_6.npy
Saved: ./position_files\good_right\63692_5.npy
Saved: ./position_files\good_right\79818_1.npy
Saved: ./position_files\good_right\79863_4.npy
Saved: ./position_files\good_right\80125_2.npy
Saved: ./position_files\good_right\80282_2.npy
Saved: ./position_files\good_right\80368_1.npy
Saved: ./position_files\good_right\80434_1.npy
Saved: ./position_files\good_right\80532_1.npy
Saved: ./position_files\good_right\80561_3.npy
Saved: ./position_files\good_right\80638_4.npy
Saved: ./position_files\good_right\80661_3.npy
Saved: ./posi

In [ ]:
import os
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Paths
input_dir = './position_files'

X_list = []
y_list = []

# Read each labeled subdirectory
for label in os.listdir(input_dir):
    label_path = os.path.join(input_dir, label)
    if not os.path.isdir(label_path):
        continue

    for fname in os.listdir(label_path):
        if fname.endswith('.npy'):
            path = os.path.join(label_path, fname)
            data = np.load(path)  # shape: (frames, 34)
            X_list.append(data)
            y_list.append(label)

# Pad sequences to the same length
X_padded = pad_sequences(X_list, padding='post', dtype='float32')  # (num_samples, max_timesteps, 34)

# Encode labels into integers
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y_list)  # e.g., "good_form" → 0, etc.

# Split dataset
X_train, X_val, y_train, y_val = train_test_split(X_padded, y_encoded, test_size=0.2, random_state=42)

print("X shape:", X_train.shape)
print("y shape:", y_train.shape)
print("Classes:", label_encoder.classes_)


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Masking, BatchNormalization

timesteps = X_train.shape[1]
features = X_train.shape[2]
num_classes = 3  # you said 3 outputs

model = Sequential([
    Masking(mask_value=0.0, input_shape=(timesteps, features)),
    LSTM(128, return_sequences=True),
    BatchNormalization(),
    Dropout(0.3),
    LSTM(64),
    BatchNormalization(),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(num_classes, activation='softmax')
])


In [ ]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=50,
    batch_size=4
)

model.save("ohp_classifier_lstm.h5")